In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix

#from roboflow import Roboflow
import numpy as np
import matplotlib.pyplot as plt
import itertools
import torch

In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    # Define device
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    # If GPU is not available, fall back to CPU
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is not available. Using CPU.


### Download dataset

In [3]:
'''rf = Roboflow(api_key="gJvPdZaOYF8eb2ctWoWS")
project = rf.workspace("ai-clinic-beekeeping").project("beekeeping_step2")
version = project.version(4)
dataset = version.download("folder")'''

'rf = Roboflow(api_key="gJvPdZaOYF8eb2ctWoWS")\nproject = rf.workspace("ai-clinic-beekeeping").project("beekeeping_step2")\nversion = project.version(4)\ndataset = version.download("folder")'

### PREPARE DATASET

In [4]:
dataset="beekeeping_step2-4"
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset+'/train',  # This is the target directory
    target_size=(410, 410),  # All images will be resized to 410x410
    batch_size=32,
    class_mode='binary')  # Since we use binary_crossentropy loss, we need binary labels

validation_generator = valid_datagen.flow_from_directory(
    dataset+'/valid',
    target_size=(410, 410),
    batch_size=32,
    class_mode='binary')


Found 983 images belonging to 2 classes.
Found 368 images belonging to 2 classes.


In [5]:
import os
import keras_tuner as kt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from PIL import Image

print("PIL imported successfully")

# Define the model building function
def build_model(hp):
    model = Sequential()
    kernel_size_choice = hp.Choice('kernel_size', values=[3, 5])
    kernel_size = (kernel_size_choice, kernel_size_choice)
    activation_choice = hp.Choice('activation', values=['relu', 'tanh'])
    
    # Adding filter choices for each Conv2D layer
    filters_1 = hp.Choice('filters_1', values=[16, 32, 64])
    filters_2 = hp.Choice('filters_2', values=[32, 64, 128])
    filters_3 = hp.Choice('filters_3', values=[64, 128, 256])

    model.add(Input(shape=(410, 410, 3)))  # Use Input layer here
    model.add(Conv2D(filters_1, kernel_size, activation=activation_choice))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters_2, kernel_size, activation=activation_choice))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters_3, kernel_size, activation=activation_choice))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation_choice))
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
    
    model.compile(
        optimizer=Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

# Define the dataset paths
dataset = "beekeeping_step2-4"

# ImageDataGenerator instances
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    dataset+'/train',  # This is the target directory
    target_size=(410, 410),  # All images will be resized to 410x410
    batch_size=32,
    class_mode='binary')  # Since we use binary_crossentropy loss, we need binary labels

validation_generator = valid_datagen.flow_from_directory(
    dataset+'/valid',
    target_size=(410, 410),
    batch_size=32,
    class_mode='binary')

# Ensure the tuner directory is clean
tuner_dir = 'my_dir'
if os.path.exists(tuner_dir):
    import shutil
    shutil.rmtree(tuner_dir)

# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    hyperband_iterations=1,
    directory=tuner_dir,
    project_name='beekeeping')

# Define the early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True)

# Run the hyperparameter search
try:
    print("Starting hyperparameter search...")
    tuner.search(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        callbacks=[early_stopping])
except Exception as e:
    print("Error during hyperparameter tuning:", e)

# Get the optimal hyperparameters
try:
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"""
    The hyperparameter search is complete. The optimal parameters are:
    Activation function: {best_hps.get('activation')}
    Kernel size: {best_hps.get('kernel_size')}
    Filters for Conv2D layers: 
        Layer 1: {best_hps.get('filters_1')}
        Layer 2: {best_hps.get('filters_2')}
        Layer 3: {best_hps.get('filters_3')}
    """)
except Exception as e:
    print("Error retrieving best hyperparameters:", e)


ModuleNotFoundError: No module named 'keras_tuner'

### Model Definition

In [ ]:
model.summary()

## Evaluation
### Confusion Matrix

In [ ]:
test_generator = valid_datagen.flow_from_directory(
    dataset+'/test',  # specify your test dataset directory
    target_size=(410, 410),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

predictions = model.predict(test_generator)
y_pred = np.round(predictions).astype(int).flatten()
y_true = test_generator.classes

In [ ]:
# Function to plot the confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Calculate and plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,8))
plot_confusion_matrix(cm, classes=['Non-Pollen', 'Pollen'], normalize=False, title='Confusion matrix')

### Visualize the training and validation loss and accuracy

# Summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()